In [65]:
from scipy.stats import random_correlation
import numpy as np
import scipy.stats as stats

def generate_random_numbers(n):
    if n <= 0:
        raise ValueError("n must be a positive integer")
    numbers = np.random.random(n)    
    numbers = numbers / np.sum(numbers) * (n)
    return numbers

n = 7
random_numbers = generate_random_numbers(n)
print(random_numbers)
print("Sum:", np.sum(random_numbers))

[1.60456741 0.0940027  1.72436855 1.71085905 1.54082385 0.16649924
 0.1588792 ]
Sum: 6.999999999999999


In [96]:
rng = np.random.default_rng()
covmat_Z = random_correlation.rvs(random_numbers, random_state=rng)
print(covmat_Z.shape)
mvnorm = stats.multivariate_normal(mean=np.zeros(covmat_Z.shape[0]), cov=covmat_Z) 
mvnorm

(7, 7)


In [75]:
import numpy as np
import pandas as pd

def gen_mean_std():
    return np.round(np.random.uniform(-1, 1), 2) , np.round(np.random.uniform(-1, 1), 2)

In [93]:
m1 = stats.norm(*gen_mean_std())
m2 = stats.norm(*gen_mean_std())
m3 = stats.norm(*gen_mean_std())
m4 = stats.norm(*gen_mean_std())
m5 = stats.uniform()
m6 = stats.uniform()
m7 = stats.uniform()

p_a = np.round(np.random.uniform(0.1, 0.9), 1)
tau_a = 1 - p_a

p_b = np.round(np.random.uniform(0.1, 0.9), 1)
tau_b = 1-p_b

p_y = np.round(np.random.uniform(0.1, 0.9), 1)
tau_y = 1 - p_y 

for i in range(1):
    z = mvnorm.rvs(100000)
    u = stats.norm.cdf(z)
    x1 = m1.ppf(u[:, 0])
    x2 = m2.ppf(u[:, 1])
    x3 = m3.ppf(u[:, 2])
    x4 = m4.ppf(u[:, 3])
    x5 = m5.ppf(u[:, 4])
    x6 = m6.ppf(u[:, 5])
    x7 = m7.ppf(u[:, 6])
    a = np.zeros(len(x5), dtype=int)
    a[x5 >= tau_a] = 1
    b = np.zeros(len(x6), dtype=int)
    b[x6 >= tau_b] = 1
    y = np.zeros(len(x7), dtype=int)
    y[x7 >= tau_y] = 1
    data = pd.DataFrame({'X1':x1, 'X2':x2, 'X3':x3, 'X4':x4, 'A':a, 'B':b, 'Y':y})
#     data.to_csv('synthetic_data_v2010_' + str(i) + '.csv', index=False) 
    print(data.head())

         X1        X2        X3        X4  A  B  Y
0 -1.579340 -0.784797  0.606719  1.151511  0  0  1
1 -1.328865 -0.741596  0.463265  0.612546  0  0  0
2 -1.357357 -0.636942  0.228926  0.166021  0  1  0
3 -0.936441 -0.735279  0.559075  0.443410  0  0  0
4 -1.994651 -0.637910 -0.179197  1.050773  0  0  1


In [9]:
import numpy as np
import pandas as pd
from scipy.stats import random_correlation
import scipy.stats as stats
from tqdm import tqdm

def generate_n_random_numbers_with_sum_n(n):
    if n <= 0:
        raise ValueError("n must be a positive integer")
    numbers = np.random.random(n)    
    numbers = numbers / np.sum(numbers) * (n)
    return numbers

def gen_mean_std():
    return np.round(np.random.uniform(-1, 1), 2) , np.round(np.random.uniform(0, 1), 2)

def generate_simulated_data(n=10000, n_datasets=100, num_non_sensitive=4, num_sensitive=2, latent_dim=None, non_sensitive_dist='normal'):
    """
    Generate a list of simulated datasets with both non-sensitive and sensitive variables.

    Parameters:
    - n (int): The number of samples in each dataset. Default is 10,000.
    - n_datasets (int): The number of datasets to sample. Default is 100.
    - num_non_sensitive (int): The number of non-sensitive variables in each dataset. Default is 4.
    - num_sensitive (int): The number of sensitive variables in each dataset. Default is 2.
    - non_sensitive_dist (str): Distribution type for non-sensitive variables. Options are:
      - 'normal': Normal distribution
      - 'exp': Exponential distribution
      - 'gamma': Gamma distribution
      - 'beta': Beta distribution
      - 'lognorm': Log-normal distribution
      Default is 'normal'.

    Returns:
    - List[pd.DataFrame]: A list containing `n_datasets` DataFrames, each representing a simulated dataset with the following columns:
      - Non-sensitive variables: `X1`, `X2`, ..., up to `X{num_non_sensitive}`
      - Sensitive variables: `S1`, `S2`, ..., up to `S{num_sensitive}`
      - Binary outcome variable: `Y`

    Raises:
    - ValueError: If an unrecognized distribution is provided for `non_sensitive_dist`.

    Notes:
    - The function generates a multivariate normal distribution for the latent variables and transforms them to produce the non-sensitive and sensitive variables.
    - Thresholds for sensitive variables and the binary outcome variable are determined based on uniform distributions and applied to the generated data.

    Example:
    >>> datasets = generate_simulated_data(n=5000, n_datasets=10, num_non_sensitive=3, num_sensitive=2, non_sensitive_dist='gamma')
    >>> len(datasets)
    10
    >>> datasets[0].head()
         X1        X2        X3  S1  S2  Y
    0  ...       ...       ...   0   1  0
    1  ...       ...       ...   1   0  1
    """
    total_vars = num_non_sensitive + num_sensitive + 1
    random_numbers = generate_n_random_numbers_with_sum_n(total_vars)
    rng = np.random.default_rng()
    covmat_Z = random_correlation.rvs(random_numbers, random_state=rng)
    mvnorm = stats.multivariate_normal(mean=np.zeros(covmat_Z.shape[0]), cov=covmat_Z) 

    non_sensitive_dists = []
    for i in range(num_non_sensitive):
        if non_sensitive_dist == 'normal':
            non_sensitive_dists.append(stats.norm(*gen_mean_std()))
        elif non_sensitive_dist == 'exp':
            scale = 1  
            non_sensitive_dists.append(stats.expon(scale=scale))
        elif non_sensitive_dist == 'gamma':
            shape = 2.0
            scale = 1.0
            non_sensitive_dists.append(stats.gamma(shape, scale=scale))
        elif non_sensitive_dist == 'beta':
            a = 2.0
            b = 5.0
            non_sensitive_dists.append(stats.beta(a,b))
        elif non_sensitive_dist == 'lognorm':
            shape = 0.954
            scale = 1.0
            non_sensitive_dists.append(stats.lognorm(s=shape, scale=scale))
        else:
            raise ValueError(f"Unrecognized distribution: {non_sensitive_dist}")

    sensitive_dists = []
    for i in range(num_sensitive):
        sensitive_dists.append(stats.uniform())

    distY = stats.uniform()
    
    tau_sensitive = []
    for i in range(num_sensitive):
        p = np.round(np.random.uniform(0.1, 0.9), 1)
        tau_sensitive.append(1 - p)

    p_y = np.round(np.random.uniform(0.1, 0.9), 1)
    tau_y = 1 - p_y 
    
    datasets = []
    
    for i in tqdm(range(n_datasets)):
        z = mvnorm.rvs(n)
        u = stats.norm.cdf(z)
        
        X = []
        for i in range(num_non_sensitive):
            X.append(non_sensitive_dists[i].ppf(u[:, i]))
            
        S = []
        for i in range(num_sensitive):
            sensitive_data = sensitive_dists[i].ppf(u[:, num_non_sensitive + i])
            sensitive_data_bin = np.zeros(len(sensitive_data), dtype=int)
            sensitive_data_bin[sensitive_data >= tau_sensitive[i]] = 1
            S.append(sensitive_data_bin)
        xY = distY.ppf(u[:, num_non_sensitive + num_sensitive])
        y = np.zeros(len(xY), dtype=int)
        y[xY >= tau_y] = 1     
        
        datasets.append(pd.DataFrame({f'X{i+1}': X[i] for i in range(len(X))} | {f'S{i+1}': S[i] for i in range(len(S))} | {'Y':y}))

    return datasets

In [10]:
datasets = generate_simulated_data(n=10000, n_datasets=10, num_non_sensitive=4, num_sensitive=1, latent_dim=None, non_sensitive_dist='normal')

100%|██████████| 10/10 [00:00<00:00, 71.51it/s]


In [11]:
datasets[6].Y.mean()

0.5057

In [12]:
for i in datasets:
    print(i.Y.mean())

0.4993
0.4927
0.4995
0.5061
0.504
0.5022
0.5057
0.5025
0.5027
0.4959


In [13]:
for i in datasets:
    print(i.S1.mean())

0.5959
0.5992
0.6011
0.5978
0.5919
0.6017
0.5955
0.6024
0.5998
0.6


In [34]:
from scipy import stats
import numpy as np
import pandas as pd
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

In [35]:
import matplotlib.pyplot as plt
import seaborn as sns

In [36]:
class Encoder(nn.Module):
    def __init__(self, input_dim):
        super(Encoder, self).__init__()
        self.fc1 = nn.Linear(input_dim, input_dim-1)
        self.relu = nn.ReLU()
    def forward(self, x):
#         return self.relu(self.fc1(x))
        return self.fc1(x)

In [37]:
class Predictor(nn.Module):
    def __init__(self, input_dim):
        super(Predictor, self).__init__()
        self.fc1 = nn.Linear(input_dim, 1)
        self.sigmoid = nn.Sigmoid()
    def forward(self, x):
        return self.sigmoid(self.fc1(x))

In [38]:
class Adversary(nn.Module):
    def __init__(self, input_dim):
        super(Adversary, self).__init__()
        self.fc1 = nn.Linear(input_dim, 1)
        self.sigmoid = nn.Sigmoid()
    def forward(self, x):
        return self.sigmoid(self.fc1(x))

In [39]:
class CustomDataset(Dataset):
    def __init__(self, dataframe):
        self.X = torch.tensor(dataframe[['X1', 'X2', 'X3', 'X4', 'S1']].values, dtype=torch.float32)
#         self.A = torch.tensor(dataframe[['A']].values, dtype=torch.float32)
#         self.B = torch.tensor(dataframe[['B']].values, dtype=torch.float32)
        self.Y = torch.tensor(dataframe[['Y']].values, dtype=torch.float32)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
#         return self.X[idx], self.A[idx], self.B[idx], self.Y[idx]
        return self.X[idx], self.Y[idx]

dataset = CustomDataset(datasets[0])
dataloader = DataLoader(dataset, batch_size=256, shuffle=True)

In [115]:
encoder = Encoder(5)
predictor = Predictor(4)
adversary = Adversary(4)

In [116]:
learning_rate = 0.01

criterion_enc = nn.MSELoss()
criterion_pred = nn.BCELoss()
# criterion_adv = nn.BCELoss()
criterion_adv = nn.L1Loss()

optimizer_pred = optim.Adam(predictor.parameters(), lr=learning_rate)
optimizer_enc = optim.Adam(encoder.parameters(), lr=learning_rate)
# optimizer = optim.Adam(list(predictor.parameters()), lr=learning_rate)
# optimizer = optim.Adam(list(encoder.parameters()) + list(predictor.parameters()), lr=learning_rate)
optimizer_adv = optim.Adam(list(adversary.parameters()), lr=learning_rate)

In [117]:
for a,b in dataloader:
    print(a.shape)
    print(b.shape)
    break

torch.Size([256, 5])
torch.Size([256, 1])


In [118]:
num_epochs = 50
encoder.train()
predictor.train()
adversary.train()

Adversary(
  (fc1): Linear(in_features=4, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)

In [122]:
for epoch in range(num_epochs):
    
    for data, labels in dataloader:
        A = data[:,-1].view(-1,1).float()
        len_A0Y0 = ((A == 0) & (labels == 0)).sum().item()
        len_A0Y1 = ((A == 0) & (labels == 1)).sum().item()
        len_A1Y0 = ((A == 1) & (labels == 0)).sum().item()
        len_A1Y1 = ((A == 1) & (labels == 1)).sum().item()
        
        total_length = len_A0Y0 + len_A0Y1 + len_A1Y0 + len_A1Y1

        A0Y0 = len_A0Y0 / total_length
        A0Y1 = len_A0Y1 / total_length
        A1Y0 = len_A1Y0 / total_length
        A1Y1 = len_A1Y1 / total_length
        
        AY_proportion = [[A0Y0, A0Y1], [A1Y0, A1Y1]]
        A_prop = [AY_proportion[0][0] + AY_proportion[0][1], AY_proportion[1][0] + AY_proportion[1][1]]
        Y_prop = [AY_proportion[0][0] + AY_proportion[1][0], AY_proportion[0][1] + AY_proportion[1][1]]

        wts = A_prop[0] * (1 - A) + A_prop[1] * A
        print(f"len_A0Y0: {len_A0Y0}")
        print(f"len_A0Y1: {len_A0Y1}")
        print(f"len_A1Y0: {len_A1Y0}")
        print(f"len_A1Y1: {len_A1Y1}")
        print(f"Sum of all lengths: {total_length}")
        print(f"AY_proportion: {AY_proportion}")
        print(f"A_prop: {A_prop}")
        print(f"Y_prop: {Y_prop}")
        print(f"wts: {wts.shape}")
        
        torch.autograd.set_detect_anomaly(True)
        
        x_recon = encoder(data)
        y_pred = predictor(x_recon)
        adv_pred = adversary(x_recon)
        
        recon_loss = criterion_enc(x_recon, data[:,:4])
        
#         class_loss = class_coeff * criterion_pred(y_pred, labels)
        class_loss = 5 * criterion_pred(y_pred, labels)
#         aud_loss = -fair_coeff * criterion_adv(A, A_logits)
        aud_loss = -5 * criterion_adv(adv_pred, A)
    
        weighted_aud_loss = torch.mean(aud_loss * torch.squeeze(wts))

        loss = recon_loss + class_loss + weighted_aud_loss
        
        optimizer.zero_grad()
        loss.backward(retain_graph=True)
        torch.nn.utils.clip_grad_norm_(list(encoder.parameters())+list(predictor.parameters()), 5.0)
        optimizer.step()
        aud_steps = 10
        for i in range(aud_steps):
            optimizer_adv.zero_grad()
            if i != aud_steps - 1:
                loss.backward(retain_graph=True)
            else:
                loss.backward()
            torch.nn.utils.clip_grad_norm_(adversary.parameters(), 5.0)
            optimizer_adv.step()
        break
    break

len_A0Y0: 46
len_A0Y1: 55
len_A1Y0: 75
len_A1Y1: 80
Sum of all lengths: 256
AY_proportion: [[0.1796875, 0.21484375], [0.29296875, 0.3125]]
A_prop: [0.39453125, 0.60546875]
Y_prop: [0.47265625, 0.52734375]
wts: torch.Size([256, 1])


/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/torch/autograd/__init__.py:251: UserWarning: Error detected in AddmmBackward0. Traceback of forward call that caused the error:
  File "/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/ipykernel/__main__.py", line 5, in <module>
    app.launch_new_instance()
  File "/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 739, in start
    self.io_loop.start()
  File "/home/ec2-user/anaconda3/envs/pytorch_p310/li

RuntimeError: one of the variables needed for gradient computation has been modified by an inplace operation: [torch.FloatTensor [4, 1]], which is output 0 of AsStridedBackward0, is at version 4002; expected version 4001 instead. Hint: the backtrace further above shows the operation that failed to compute its gradient. The variable in question was changed in there or anywhere later. Good luck!

In [124]:
# num_epochs=100
# for epoch in range(num_epochs):
    
#     for data, labels in dataloader: 
#         adversary.zero_grad()
#         x_recon = encoder(data)
#         adv_pred = adversary(x_recon)
#         la = criterion_adv(adv_pred, data[:,4].view(-1,1).float())
#         la.backward()
        
#         optimizer_adv.step()
        
#     for data, labels in dataloader: 
#         pass
    
#     encoder.zero_grad()
#     predictor.zero_grad()
#     x_recon = encoder(data)
#     y_pred = predictor(x_recon)
#     adv_pred = adversary(x_recon)

#     lx = criterion_enc(x_recon, data[:,:4])
#     lp = criterion_pred(y_pred, labels)
#     la = criterion_adv(adv_pred, data[:,4].view(-1,1).float())
    
#     combined_loss = lx + lp - la
#     combined_loss.backward()

#     optimizer_pred.step()
#     optimizer_enc.step()

#     print(f'Epoch [{epoch+1}/{num_epochs}], Loss Adv: {la.item():.4f}, Loss Recon: {lx.item():.4f}, Loss P: {lp.item():.4f}, Loss Comb: {combined_loss.item():.4f}')

In [120]:
index = 0
df_original = datasets[index].copy()

dataset = CustomDataset(df_original)
dataloader = DataLoader(dataset, batch_size=256, shuffle=False)

encoder_outputs = []
encoder.eval()
with torch.no_grad():
    for batch_X, _ in dataloader:
        output = encoder(batch_X)
        encoder_outputs.append(output)
encoder_outputs = torch.cat(encoder_outputs, dim=0)
print(encoder_outputs.shape)

encoder_outputs_np = encoder_outputs.numpy()
df = pd.DataFrame(encoder_outputs_np, columns=['X1', 'X2', 'X3', 'X4'])

df_A_B = df_original[['S1', 'Y']].reset_index(drop=True)
df = pd.concat([df, df_A_B], axis=1)

correlation_matrix = df.corr()

torch.Size([10000, 4])


In [123]:
# plt.figure(figsize=(20, 8))
# plt.subplot(1,2,1)
# sns.heatmap(datasets[0].corr(), annot=True, cmap='coolwarm', fmt='.4f', vmin=-1, vmax=1)
# plt.subplot(1,2,2)
# sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.4f', vmin=-1, vmax=1)
# plt.title('Pearson Correlation Heatmap')
# plt.show()